In [1]:
import numpy as np
import pandas as pd
import optuna
from scipy.sparse import csr_matrix
from weighting_strategies import (
    bm25_weight, tfidf_weight, normalized_weight,
    log_weight, confidence_weight, power_weight,
    pmi_weight, robust_user_centric_weight, sigmoid_propensity_weight, power_lift_weight, robust_user_centric_weight_v2
)
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import train_test_split, precision_at_k, ndcg_at_k


In [2]:
import sys
import os

# Add the parent directory to sys.path to resolve imports from sibling directories
sys.path.append(os.path.abspath(".."))

from utils.sparse import transform_dataframe_to_sparse

In [3]:
lastfm_df = (
    pd.read_csv(
        "/home/coder/projects/rec-sys-research/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv",
        sep="\t",
        header=None,
        usecols=[0, 2, 3],
        names=['user_id', 'item_id', 'play_count'],
    )
    .loc[:, ['user_id', 'item_id', 'play_count']]
    .dropna()
    .rename(columns={'play_count': 'target'})
)
lastfm_df['user_id'].nunique(), lastfm_df['item_id'].nunique(), lastfm_df.shape[0]

(358868, 292363, 17535451)

In [4]:
user_item_matrix, user_mapping, item_mapping = transform_dataframe_to_sparse(
    lastfm_df, row_field='user_id', col_field='item_id', data_field='target'
)


train_val_mat, test_mat = train_test_split(user_item_matrix, train_percentage=0.9, random_state=42)
train_mat, val_mat = train_test_split(train_val_mat, train_percentage=0.9, random_state=42)

print(f"Train Shape: {train_mat.shape}, Val Shape: {val_mat.shape}, Test Shape: {test_mat.shape}")

Train Shape: (358868, 292363), Val Shape: (358868, 292363), Test Shape: (358868, 292363)


In [ ]:
results_folder = "results/lastfm_360k"
results_filename = "lastfm_360k_experiment_results.csv"

import time

def run_hyperparameter_optimization(
    train_mat: csr_matrix,
    val_mat: csr_matrix,
    train_val_mat: csr_matrix,
    test_mat: csr_matrix,
    weighting_strategy: str,
    algorithm: str,
    n_trials: int = 20,
    output_dir: str = None,
) -> pd.DataFrame:
    results = []
    algorithms = {
        "ALS_factors=10": lambda: AlternatingLeastSquares(factors=10, random_state=42),
        "ALS_factors=100": lambda: AlternatingLeastSquares(factors=100, random_state=42),
    }
    strategies = [
        "no_weighting",
        "bm25",
        "tfidf",
        "log", 
        "confidence",
        "power",
        "normalized",
        "pmi",
        "robust_user_centric",
        "robust_user_centric_weight_v2",
        "sigmoid_propensity",
        "power_lift"
    ]
    if weighting_strategy not in strategies:
        raise ValueError(f"Weighting strategy '{weighting_strategy}' is not recognized.")
    strategy = weighting_strategy

    if algorithm not in algorithms:
        raise ValueError(f"Algorithm '{algorithm}' is not recognized.")
    algo_name = algorithm
    AlgoFactory = algorithms[algorithm]

    print(f"Running optimization for {algo_name} with {strategy}...")

    def get_weighted_matrix(matrix, params):
        weighted = matrix.copy()
        if strategy == "bm25":
            weighted = bm25_weight(weighted, K1=params.get("bm25_k1"), B=params.get("bm25_b"))
        elif strategy == "confidence":
            weighted = confidence_weight(weighted, alpha=params.get("conf_alpha"))
        elif strategy == "power":
            weighted = power_weight(weighted, p=params.get("power_p"))
        elif strategy == "tfidf":
            weighted = tfidf_weight(weighted)
        elif strategy == "log":
            weighted = log_weight(weighted)
        elif strategy == "normalized":
            weighted = normalized_weight(weighted)
        elif strategy == "pmi":
            weighted = pmi_weight(weighted)
        elif strategy == "robust_user_centric":
            weighted = robust_user_centric_weight(weighted, scale_factor=params.get("scale_factor"))
        elif strategy == "sigmoid_propensity":
            weighted = sigmoid_propensity_weight(weighted, p=params.get("p"), beta=params.get("beta"))
        elif strategy == "power_lift":
            weighted = power_lift_weight(weighted, p=params.get("p"))
        elif strategy == "robust_user_centric_weight_v2":
            weighted = robust_user_centric_weight_v2(weighted, lower_q=params.get("lower_q"), upper_q=params.get("upper_q"))
        return weighted

    def objective(trial):
        params = {}
        # Suggest weighting strategy parameters
        if strategy == "bm25":
            params["bm25_k1"] = trial.suggest_float("bm25_k1", 0.1, 1000)
            params["bm25_b"] = trial.suggest_float("bm25_b", 0.0, 1.0)
        elif strategy == "confidence":
            params["conf_alpha"] = trial.suggest_float("conf_alpha", 1.0, 150.0)
        elif strategy == "power":
            params["power_p"] = trial.suggest_float("power_p", 0.1, 1.5)
        elif strategy == "robust_user_centric":
            params["scale_factor"] = trial.suggest_float("scale_factor", 0.1, 10.0)
        elif strategy == "robust_user_centric_weight_v2":
            params["lower_q"] = trial.suggest_float("lower_q", 5.0, 45.0)
            params["upper_q"] = trial.suggest_float("upper_q", 55.0, 95.0)
        elif strategy == "sigmoid_propensity":
            params["p"] = trial.suggest_float("p", 0.1, 5.0)
            params["beta"] = trial.suggest_float("beta", 0.0, 1.0)
        elif strategy == "power_lift":
            params["p"] = trial.suggest_float("p", 0.1, 1.5)
        weighted_train = get_weighted_matrix(train_mat, params)
        
        # Train Model
        model = AlgoFactory()
        model.fit(weighted_train, show_progress=False)

        # Evaluate on Validation Set
        return ndcg_at_k(model, train_mat, val_mat, K=20, show_progress=False)

    # Optimize only if strategy has parameters
    current_trials = n_trials if strategy in ["bm25", "confidence", "power", "robust_user_centric", "robust_user_centric_weight_v2", "sigmoid_propensity", "power_lift"] else 1
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=current_trials, n_jobs=-1)

    # --- Final Retraining & Testing ---
    # Use best params to weight the full train_val matrix
    best_params = study.best_params
    weighted_train_val = get_weighted_matrix(train_val_mat, best_params)

    # Train Final Model
    final_model = AlgoFactory()
    
    start_time = time.time()
    final_model.fit(weighted_train_val, show_progress=False)
    end_time = time.time()
    
    # Evaluate on Test Set
    test_ndcg_10 = ndcg_at_k(final_model, train_val_mat, test_mat, K=10, show_progress=False)
    test_precision_10 = precision_at_k(final_model, train_val_mat, test_mat, K=10, show_progress=False)
    test_ndcg_20 = ndcg_at_k(final_model, train_val_mat, test_mat, K=20, show_progress=False)
    test_precision_20 = precision_at_k(final_model, train_val_mat, test_mat, K=20, show_progress=False)

    results.append({
        "Algorithm": algo_name,
        "Strategy": strategy,
        "Number of Optimization Trials": current_trials,
        "Best Val NDCG@20": study.best_value,
        "Test NDCG@10": test_ndcg_10,
        "Test NDCG@20": test_ndcg_20,
        "Test Precision@10": test_precision_10,
        "Test Precision@20": test_precision_20,
        "Final Train Time (s)": end_time - start_time,
        "Best Params": best_params
    })

    if output_dir:
        output_path = os.path.join(output_dir, f"{algo_name}_{strategy}_results.csv")
        pd.DataFrame(results).to_csv(output_path, index=False)
    return pd.DataFrame(results)

In [ ]:
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

# Execution of all experiments
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="confidence", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="confidence", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-01-30 09:25:14,851] A new study created in memory with name: no-name-af5ef919-2b93-421d-a4fd-849484d241b9


Running optimization for ALS_factors=10 with no_weighting...


[I 2026-01-30 09:25:25,418] Trial 0 finished with value: 0.09808161938868308 and parameters: {}. Best is trial 0 with value: 0.09808161938868308.
[I 2026-01-30 09:25:42,413] A new study created in memory with name: no-name-499a5779-4433-46d4-87e8-647884361337


Running optimization for ALS_factors=10 with bm25...


[I 2026-01-30 09:26:21,753] Trial 1 finished with value: 0.09645463137925213 and parameters: {'bm25_k1': 225.48082597129576, 'bm25_b': 0.5287138717868045}. Best is trial 1 with value: 0.09645463137925213.
[I 2026-01-30 09:26:21,814] Trial 3 finished with value: 0.11396118964270216 and parameters: {'bm25_k1': 23.043610429295562, 'bm25_b': 0.08553331531117592}. Best is trial 3 with value: 0.11396118964270216.
[I 2026-01-30 09:26:22,187] Trial 0 finished with value: 0.09083202639244789 and parameters: {'bm25_k1': 390.3740339836404, 'bm25_b': 0.8429585315007274}. Best is trial 3 with value: 0.11396118964270216.
[I 2026-01-30 09:26:22,662] Trial 2 finished with value: 0.08696873586782118 and parameters: {'bm25_k1': 730.9996598439025, 'bm25_b': 0.3629667005323126}. Best is trial 3 with value: 0.11396118964270216.
[I 2026-01-30 09:27:00,568] Trial 4 finished with value: 0.08377728399514059 and parameters: {'bm25_k1': 999.0718547570991, 'bm25_b': 0.9304015463815763}. Best is trial 3 with value

...

Warning NaN Detected in row 192 of 292363
Warning NaN Detected in row 10528 of 292363
Warning NaN Detected in row 250 of 292363
Warning NaN Detected in row 269 of 292363
Warning NaN Detected in row 20739 of 292363
Warning NaN Detected in row 10508 of 292363
Warning NaN Detected in row 10510 of 292363
Warning NaN Detected in row 51415 of 292363
Warning NaN Detected in row 174347 of 292363
Warning NaN Detected in row 245997 of 292363
Warning NaN Detected in row 194784 of 292363
Warning NaN Detected in row 31232 of 292363
Warning NaN Detected in row 11667 of 292363
Warning NaN Detected in row 185783 of 292363
Warning NaN Detected in row 103918 of 292363
Warning NaN Detected in row 186446 of 292363
Warning NaN Detected in row 12833 of 292363
Warning NaN Detected in row 4072 of 292363
Warning NaN Detected in row 45045 of 292363
Warning NaN Detected in row 169640 of 292363
Warning NaN Detected in row 27493 of 292363
Warning NaN Detected in row 27837 of 292363
Warning NaN Detected in row 2330

[I 2026-01-30 12:05:41,156] Trial 15 finished with value: 0.13636529567094358 and parameters: {'p': 0.12422404133762173}. Best is trial 11 with value: 0.15389283625134473.


Warning NaN Detected in row 302 of 292363
Warning NaN Detected in row 10494 of 292363
Warning NaN Detected in row 253 of 292363
Warning NaN Detected in row 20736 of 292363
Warning NaN Detected in row 124425 of 292363
Warning NaN Detected in row 37135 of 292363
Warning NaN Detected in row 67972 of 292363
Warning NaN Detected in row 16816 of 292363
Warning NaN Detected in row 277 of 292363
Warning NaN Detected in row 10520 of 292363
Warning NaN Detected in row 253 of 292363
Warning NaN Detected in row 64558 of 292363
Warning NaN Detected in row 46075 of 292363
Warning NaN Detected in row 117906 of 292363
Warning NaN Detected in row 26444 of 292363


[I 2026-01-30 12:06:26,586] Trial 16 finished with value: 0.14144831396653526 and parameters: {'p': 0.1440733558007744}. Best is trial 11 with value: 0.15389283625134473.
[I 2026-01-30 12:06:27,308] Trial 17 finished with value: 0.16445197183219615 and parameters: {'p': 0.3200297249072065}. Best is trial 17 with value: 0.16445197183219615.
[I 2026-01-30 12:06:30,181] Trial 18 finished with value: 0.00790366358894014 and parameters: {'p': 1.4826345150802687}. Best is trial 17 with value: 0.16445197183219615.
[I 2026-01-30 12:06:31,833] Trial 19 finished with value: 0.1621389483598752 and parameters: {'p': 0.3561350679055588}. Best is trial 17 with value: 0.16445197183219615.


,Algorithm,Strategy,Best Val NDCG@20,Test NDCG@20,Test Precision@20,Final Train Time (s),Best Params
13,ALS_factors=100,bm25,0.171900,0.193160,0.252240,11.137091,"{'bm25_k1': 3.6915452916512734, 'bm25_b': 0.80..."
16,ALS_factors=100,confidence,0.169925,0.189744,0.250750,11.147264,{'conf_alpha': 1.449770352422187}
19,ALS_factors=100,pmi,0.165601,0.185581,0.240786,11.131036,{}
23,ALS_factors=100,power_lift,0.164452,0.183018,0.238844,11.150032,{'p': 0.3200297249072065}
14,ALS_factors=100,tfidf,0.161872,0.179705,0.242950,11.146544,{}
17,ALS_factors=100,power,0.159287,0.178433,0.235119,11.150032,{'power_p': 0.5623046594177248}
22,ALS_factors=100,sigmoid_propensity,0.153111,0.174393,0.227104,11.128924,"{'p': 0.2279499441848788, 'beta': 0.9773243639..."
20,ALS_factors=100,robust_user_centric,0.152840,0.173146,0.225228,10.704933,{'scale_factor': 9.971762914520497}
15,ALS_factors=100,log,0.147436,0.167481,0.217905,11.140960,{}
12,ALS_factors=100,no_weighting,0.131322,0.142881,0.204102,11.130598,{}


In [ ]:
import glob

all_results = []
# Match any CSV in the result folder
for f in glob.glob(f"{results_folder}/*.csv"):
    all_results.append(pd.read_csv(f))

if all_results:
    experiment_results = pd.concat(all_results)
    experiment_results = experiment_results.sort_values("Test NDCG@20", ascending=False)
    experiment_results.to_csv(results_filename, index=False)
else:
    print("No results found.")

experiment_results